# ETF - Strategy Backtesting (2012-12-03 to 2017-11-30)

### Strategy:
1. At the start of each week, check the 30 week (6 months) rate of change of NiftyBees and GoldBees.
2. If none of the ETFs are returning more than 3% in 6 months. Do nothing. Be invested in LiquidBees.
3. If one or both ETFs are returning more than 3% invest your entire position in that ETF. (Higher Performance ETF).

Backtested results excluding dividends 2005-2015
* 10 Years
* CAGR: 25.22%
* Max Drawdown: 21.96%

(^ backtest results as given)

In [1]:
import pandas as pd

In [2]:
df = pd.read_excel(r'data/etf - 30 week.xlsx')

### Data Wrangling - Daily to Weekly

Data wrangling performed to compress the large dataset to retain only the essential data points (week beginning and ending).
* week ending - for calculating Rate of Change (RoC)
* week beginning - for trade actions (buy/sell)

In [3]:
# Converting data from df to weekly format

'''
Abbreviation for columns
N - NiftyBees
G - GoldBees
L - LiquidBees
B - Begin
E - End
'''

weekly_col = ['WB','WE','NB','NE','GB','GE','LB','LE']
weekly = pd.DataFrame(columns=weekly_col)
curr_week = -1
df_prev_week = 0
# cw_data - current week data
cw_data = dict.fromkeys(weekly_col,0.0)

for row in range(len(df)):
    # cw_ds - current week dataset
    cw_ds = df.iloc[row]
    if df_prev_week == cw_ds[1]:
        cw_data['WE'] = cw_ds[0]
        cw_data['NE'] = cw_ds[3]
        cw_data['GE'] = cw_ds[4]
        cw_data['LE'] = cw_ds[5]
    else:
        weekly.loc[curr_week] = cw_data
        curr_week += 1
        cw_data['WB'] = cw_data['WE'] = cw_ds[0]
        cw_data['NB'] = cw_data['NE'] = cw_ds[3]
        cw_data['GB'] = cw_data['GE'] = cw_ds[4]
        cw_data['LB'] = cw_data['LE'] = cw_ds[5]
    df_prev_week = cw_ds[1]

weekly.loc[curr_week] = cw_data
weekly.drop([-1], inplace=True)
print("Weekly Data:")
weekly['WB'] = pd.to_datetime(weekly['WB'])
weekly['WE'] = pd.to_datetime(weekly['WE'])
print(weekly)

Weekly Data:
            WB         WE       NB       NE       GB       GE       LB  \
0   2012-12-03 2012-12-07   590.84   598.91  2991.00  2944.00  1000.00   
1   2012-12-10 2012-12-14   594.67   592.20  2961.80  2943.65  1000.00   
2   2012-12-17 2012-12-21   590.02   588.87  2947.00  2908.20   999.99   
3   2012-12-24 2012-12-28   589.23   594.78  2917.90  2905.35  1000.00   
4   2012-12-31 2013-01-04   594.57   605.65  2903.10  2892.95  1000.00   
5   2013-01-07 2013-01-11   603.21   599.27  2904.90  2895.85  1000.00   
6   2013-01-14 2013-01-18   607.22   610.52  2889.80  2900.20  1000.00   
7   2013-01-21 2013-01-25   614.37   611.44  2889.90  2890.40   999.99   
8   2013-01-28 2013-02-01   612.69   606.07  2891.65  2871.95  1000.00   
9   2013-02-04 2013-02-08   602.10   592.77  2867.45  2882.80  1000.00   
10  2013-02-11 2013-02-15   594.64   592.68  2883.75  2855.70   999.99   
11  2013-02-18 2013-02-22   593.80   593.15  2831.60  2778.60   999.99   
12  2013-02-25 2013-03-01

### Trade Simulation
Simulation of Trade is performed with the following conditions:
* Period of Study: 2012-12-03 to 2017-11-30
* Initial Funds: ₹10,000
* Interval for RoC calculation: 30 weeks ~ 6 months

Scrip:
* N - NiftyBees
* G - GoldBees
* L - LiquidBees

In [4]:
initial_funds = 10000.00
fund_bal = initial_funds
trade_book = pd.DataFrame(columns = ['Date','Scrip','Value','Quantity','Amount'])
# trade book entry
tb_entry = dict.fromkeys(['Date','Scrip','Value','Quantity','Amount'])
period = 30
nifty_roc = gold_roc = 0
entry_num = 0

def roc_calc(col):
    initial = weekly.loc[week_num - period][col]
    final = weekly.loc[week_num - 1][col]
    return (final - initial) * 100 / initial

def invest_in(scrip):
    
    def entry_scrip():
        global entry_num, fund_bal
        tb_entry['Date'] = weekly.WB.values[week_num]
        tb_entry['Scrip'] = scrip
        tb_entry['Value'] = weekly[scrip+'B'].values[week_num]
        tb_entry['Quantity'] = int(fund_bal / tb_entry['Value'])
        tb_entry['Amount'] = tb_entry['Value'] * tb_entry['Quantity'] * -1
        fund_bal += tb_entry['Amount']
        # print(tb_entry, fund_bal)
        trade_book.loc[entry_num] = tb_entry
        entry_num += 1
    def exit_scrip():
        global entry_num, fund_bal
        tb_entry['Date'] = weekly.WB.values[week_num]
        tb_entry['Scrip'] = trade_book.Scrip.values[-1]
        tb_entry['Value'] = weekly[trade_book.Scrip.values[-1]+'B'].values[week_num]
        tb_entry['Quantity'] = trade_book.Quantity.values[-1]
        tb_entry['Amount'] = tb_entry['Value'] * tb_entry['Quantity']
        fund_bal += tb_entry['Amount']
        # print(tb_entry, fund_bal)
        trade_book.loc[entry_num] = tb_entry
        entry_num += 1
    
    if entry_num == 0:
        entry_scrip()
    elif scrip != trade_book.Scrip.values[-1]:
        exit_scrip()
        entry_scrip()
        

for week_num in range(period,len(weekly)):
    nifty_roc = roc_calc('NE')
    gold_roc = roc_calc('GE')
    # print("{:8.2f} {:8.2f}".format(nifty_roc, gold_roc))
    if nifty_roc > 3 or  gold_roc > 3:
        invest_in('N' if nifty_roc > gold_roc else 'G')
    else:
        invest_in('L')
        
print(trade_book)

         Date Scrip    Value Quantity    Amount
0  2013-07-01     L   999.99       10  -9999.90
1  2013-09-23     L  1000.00       10  10000.00
2  2013-09-23     N   596.77       16  -9548.32
3  2013-09-30     N   582.24       16   9315.84
4  2013-09-30     L  1000.00        9  -9000.00
5  2013-10-14     L  1000.00        9   9000.00
6  2013-10-14     N   617.75       15  -9266.25
7  2013-11-11     N   615.80       15   9237.00
8  2013-11-11     G  2866.55        3  -8599.65
9  2013-12-30     G  2725.00        3   8175.00
10 2013-12-30     N   635.78       14  -8900.92
11 2014-01-20     N   638.78       14   8942.92
12 2014-01-20     G  2774.00        3  -8322.00
13 2014-03-03     G  2863.70        3   8591.10
14 2014-03-03     N   628.79       15  -9431.85
15 2015-03-30     N   850.48       15  12757.20
16 2015-03-30     L  1000.00       12 -12000.00
17 2015-04-06     L  1000.00       12  12000.00
18 2015-04-06     N   867.72       14 -12148.08
19 2015-05-18     N   840.68       14  1

### Trade Analysis

Int64Index([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16,
            17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33,
            34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50,
            51, 52],
           dtype='int64')

In [6]:
ltp = weekly[trade_book.Scrip.values[-1]+'E'].values[-1]
namt = ltp * trade_book.Quantity.values[-1]
nbal = namt + fund_bal

odd_dates = pd.Series(trade_book.Date.iloc[1::2].values)
odd_dates.loc[len(odd_dates)] = weekly.WE.values[-1]
even_dates = pd.Series(trade_book.Date.iloc[::2].values)
odd_amt = pd.Series(trade_book.Amount.iloc[1::2].values)
odd_amt.loc[len(odd_amt)] = namt
even_amt = pd.Series(trade_book.Amount.iloc[::2].values)

trade_stats = pd.DataFrame()
trade_stats['EntDate'] = even_dates
trade_stats['ExtDate'] = odd_dates
trade_stats['HoldPeriod'] = odd_dates - even_dates
trade_stats['Scrip'] = pd.Series(trade_book.Scrip.iloc[::2].values)
trade_stats['EntAmt'] = even_amt
trade_stats['ExtAmt'] = odd_amt
trade_stats['Gains'] = odd_amt + even_amt
print(trade_stats)

14804.44
15049.43
      EntDate    ExtDate HoldPeriod Scrip    EntAmt    ExtAmt    Gains
0  2013-07-01 2013-09-23    84 days     L  -9999.90  10000.00     0.10
1  2013-09-23 2013-09-30     7 days     N  -9548.32   9315.84  -232.48
2  2013-09-30 2013-10-14    14 days     L  -9000.00   9000.00     0.00
3  2013-10-14 2013-11-11    28 days     N  -9266.25   9237.00   -29.25
4  2013-11-11 2013-12-30    49 days     G  -8599.65   8175.00  -424.65
5  2013-12-30 2014-01-20    21 days     N  -8900.92   8942.92    42.00
6  2014-01-20 2014-03-03    42 days     G  -8322.00   8591.10   269.10
7  2014-03-03 2015-03-30   392 days     N  -9431.85  12757.20  3325.35
8  2015-03-30 2015-04-06     7 days     L -12000.00  12000.00     0.00
9  2015-04-06 2015-05-18    42 days     N -12148.08  11769.52  -378.56
10 2015-05-18 2015-06-01    14 days     L -12000.00  12000.00     0.00
11 2015-06-01 2015-06-08     7 days     G -12255.75  12240.00   -15.75
12 2015-06-08 2015-07-20    42 days     L -12000.00  12000.